# TP2

In [ ]:
import re
import requests
import io

from IPython.display import clear_output
from bs4 import BeautifulSoup

# Déclarations des fonctions

In [ ]:
# Enlever une certains caractères dans une chaine de caractère
def rchop(thestring, ending):
    if thestring.endswith(ending):
        return thestring[:-len(ending)]
    return thestring

# Fonction d'affichage
def info(*args, **kwargs):
    rulesArg = kwargs.get("rulesArg", False)
    wordArg = kwargs.get("wordArg", True)
    resultArg = kwargs.get("resultArg", True)
    
    if(rulesArg):
        print(f'\nLes règles :')
        pprint(rules)

    if(wordArg):
        print(f'\nLe mot : {testWord}')

    if(resultArg):
        print(f'\nLes dérivations :')
        for n in result.items():
            print(str(n).replace(', [', ' -> ['))

In [ ]:
# Déclarations des variables
nature = {'Verbe': 'V', 'Nom': 'N', 'Adjectif': 'ADJ', 'Nom propre': 'NP'}
rules = []
result = {}

# Lecture du mot
print(f'Entrer le mot : ')
testWord = input()

# Effacer les outputs
clear_output(wait = False)

# Lecture de la nature du mot
print(f'Veuillez choisir la nature du mot dans la liste ci-dessous : ')
for (key, value) in nature.items():
    print(f'\t{value} pour {key}')
natureWord = input()

# Effacer les outputs
clear_output(wait = False)

# Ouverture du fichier
file = open("rules.txt", "r")

# Lecture du fichier & enregistrement des règles
if file.mode == 'r':
    lines = file.readlines()
    for line in lines:
        rules.append(line.rstrip("\n").split("==>"))

# Fermeture du fichier
file.close()

# Génération des mots et leur nature
for rule in rules:
    flag = False
    ruleG = rule[0].split(":")
    ruleD = rule[1].split(":")
    match = re.match(".*" + ruleG[1], testWord)
    if match and ruleG[0] == natureWord:
        if ruleG[1] != "":
            generatedWord = rchop(testWord, ruleG[1]) + ruleD[1]
        else:
            generatedWord = testWord + ruleD[1]
        result.setdefault(generatedWord, [])
        if(len(result[generatedWord]) != 0):
            for n in result[generatedWord]:
                if(n == ruleD[0]):
                    flag = True
            if(not flag):
                result[generatedWord].append(ruleD[0])
        else:
            result[generatedWord].append(ruleD[0])

# Fonction d'affichage (avec possibilité d'afficher que ce qui vous intéresse)
info(rulesArg = False, wordArg = False, resultArg = True)

In [ ]:
final = []
for word in result:
    # Envoie de la requete à jdm
    url = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=" + str(word) + "&rel="
    
    # Reponse de la requete
    response = requests.get(url)
    
    # bs4
    soup = BeautifulSoup(response.text, features = 'lxml')
    
    # Chercher la balise "jdm-warning"
    mydivs = soup.findAll("div", {"class": "jdm-warning"})
    
    if(not mydivs): # if jdm-warning existe (mot invalide)
        final.append(word)
    
if(final):
    print('Les mots valides sont :')
    for f in final:
        print(f'\t{f} : {result[f]}')
else:
    print('Aucun mot n\'est valide')

In [ ]:
relation_map = {'AGENT': '13', 'LIEU' : '15', 'N': '4', 'V': '4', 'ADJ': '4', 'ADV': '4'}
type_map = {'N': 'Nom:', 'V' : 'Ver:', 'ADJ' : 'Adj:'}
taliD = {}

def blabla(word, line):
    matchTestWord = re.match('e.*;.*;\'' + word + '\';.*;.*', line)
    if(matchTestWord):
        lineTestWord = line
        print(f'ICI 1 : {lineTestWord}')

for f in final:    
    for k in result[f]:
        a1 = ''
        a2 = ''
        flag = False
        print(f'Nous allons traiter le mot "{f}" qui est de nature : {k}')
        # Envoie de la requete à jdm
        if(int(relation_map[k]) == 4):
            url = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=" + str(f) + "&rel=" + str(relation_map[k])
            print(url)

            # Reponse de la requete
            response = requests.get(url)

            # bs4
            soup = BeautifulSoup(response.text, features = 'lxml')
            
            # Chercher la balise "CODE"
            mycode = soup.find('code')

            s = io.StringIO(str(mycode))
            for line in s:
                line = rchop(line, '\n')
            
                matchTestWord = re.match('e;.*;\'' + f + '\';.*;.*', line)
                if(matchTestWord):
                    lineTestWord = line
                    lineTestWordTemp = lineTestWord.split(';')
                    a1 = lineTestWordTemp[1]
                    print(f'ICI 1 : {lineTestWord}')

                matchType = re.match('e;.*;\'' + str(type_map[k]) + '\';.*;.*', line)
                if(matchType):
                    lineType = line
                    lineTypeTemp = lineType.split(';')
                    a2 = lineTypeTemp[1]
                    print(f'ICI 2 : {lineType}')

                if(a2):
                    matchRule = re.match('r;.*;' + a1 + ';' + a2 + ';4;.*', line)
                    if(matchRule):
                        lineRule = line
                        lineRuleTemp = line.split(';')
                        print(f'ICI 3 : {lineRule}')
                        a3 = int(lineRuleTemp[5])
                        if(a3 > 0):
                            taliD.setdefault(f, [])
                            if(len(taliD[f]) != 0):
                                for n in taliD[f]:
                                    if(n == k):
                                        flag = True
                                if(not flag):
                                    taliD[f].append(k)
                            else:
                                taliD[f].append(k)
        else:
            url = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=" + str(testWord) + "&rel=" + str(relation_map[k])
            print(f'{url}')

            # Reponse de la requete
            response = requests.get(url)

            # bs4
            soup = BeautifulSoup(response.text, features = 'lxml')

            # Chercher la balise "CODE"
            mycode = soup.find('code')

            s = io.StringIO(str(mycode))
            for line in s:
                line = rchop(line, '\n')

                matchTestWord = re.match('e;.*;\'' + testWord + '\';.*;.*', line)
                if(matchTestWord):
                    lineTestWord = line
                    lineTestWordTemp = lineTestWord.split(';')
                    a1 = lineTestWordTemp[1]
                    print(f'ICI 1 : {lineTestWord}')

                matchWord = re.match('e;.*;\'' + f + '\';.*;.*', line)
                if(matchWord):
                    lineWord = line
                    lineWordTemp = lineWord.split(';')
                    a2 = lineWordTemp[1]
                    print(f'ICI 2 : {lineWord}')

                if(int(relation_map[k]) == 13):
                    matchRule = re.match('r;.*;' + a1 + ';' + a2 + ';13;.*', line)
                    if(matchRule):
                        lineRule = line
                        lineRuleTemp = line.split(';')
                        print(f'ICI 3 : {lineRule}')
                        a3 = int(lineRuleTemp[5])
                        if(a3 > 0):
                            taliD.setdefault(f, [])
                            if(len(taliD[f]) != 0):
                                for n in taliD[f]:
                                    if(n == k):
                                        flag = True
                                if(not flag):
                                    taliD[f].append(k)
                            else:
                                taliD[f].append(k)

                if(int(relation_map[k]) == 15):
                    matchRule = re.match('r;.*;' + a1 + ';' + a2 + ';15;.*', line)
                    if(matchRule):
                        lineRule = line
                        lineRuleTemp = line.split(';')
                        print(f'ICI 3 : {lineRule}')
                        a3 = int(lineRuleTemp[5])
                        if(a3 > 0):
                            taliD.setdefault(f, [])
                            if(len(taliD[f]) != 0):
                                for n in taliD[f]:
                                    if(n == k):
                                        flag = True
                                if(not flag):
                                    taliD[f].append(k)
                            else:
                                taliD[f].append(k)

if(taliD):
    print('Les mots valides sont :')
    for n in taliD.items():
        temp = str(n).replace(", [", " de nature ").replace("[", "").replace("]", "").replace("'", "").replace("(", "").replace(")", "")
        print(f'\t{temp}')
else:
    print('Aucun mot n\'est valide')